In [ ]:
from ultralytics import YOLO
import cv2
import os
from datetime import datetime
import gdown

model_name = 'last20epM.pt' #Download a model from here https://drive.google.com/file/d/186m2LjW2pIzy3ImcPs41C4CvCa0_zKRN
                            #Or any other model
video_path = "1.mp4"

confidence_threshold = 0.7 # Set your confidence threshold

frame_max = 20 #amount of frames to analyze before start searching again

window_width = 640

folder_name_input = ''

#------------------------------------------------------

if folder_name_input:
    folder_name = folder_name_input
else:
    now = datetime.now()
    dt_string = now.strftime("%Y-%m-%d__%H-%M-%S")
    folder_name = f'{dt_string}__{video_path}__{model_name}'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)

if video_path.isdigit():  # Check if the input is an integer (camera index)
    video_path = int(video_path)

cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)

window_height = int(window_width*cap.get(cv2.CAP_PROP_FRAME_HEIGHT)/cap.get(cv2.CAP_PROP_FRAME_WIDTH))

cv2.namedWindow("YOLOv8 Inference", cv2.WINDOW_NORMAL)
cv2.resizeWindow("YOLOv8 Inference", window_width, window_height)

fourcc = cv2.VideoWriter_fourcc(*'H264')
out = cv2.VideoWriter(f'{folder_name}.mp4', fourcc, fps, (window_width, window_height))


frame_count = 0
frames = []
counter = 0
frames_counter =0
conf_array = []
conf_array.append(0)
conf_float = 0

#------------------------------------------------------

while True:
    _, img = cap.read()
    img_before = img.copy()
    results = model(img)
    
    annotated_frame = results[0].plot()

    for r in results:
        boxes = r.boxes
        for box in boxes:
            b = box.xyxy[0] 
            c = box.cls
            conf = box.conf
            conf_float = conf[0].cpu().numpy().item()
            conf_array.append(conf_float)

    if max(conf_array) > confidence_threshold or frame_max > frames_counter > 0:
        
        #cv2.putText(annotated_frame, f'recording frames', (330,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,255), 2)
        
        frames.append((img_before, annotated_frame, max(conf_array)))
        frames_counter += 1
        
    if frames_counter == frame_max:
        
        #cv2.putText(annotated_frame, 'frame_max', (30,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,255), 2)

        max_conf_frame = max(frames, key=lambda x: x[2])
        cv2.imwrite(os.path.join(folder_name, f'{counter:04}_1.png'), max_conf_frame[0])
        cv2.imwrite(os.path.join(folder_name, f'{counter:04}_2.png'), max_conf_frame[1])
        counter += 1
        frames = []
        conf_array = []
        conf_array.append(0)
        frames_counter = 0

    cv2.imshow('YOLOv8 Inference', annotated_frame)
    annotated_frame = cv2.resize(annotated_frame, (window_width, window_height))
    out.write(annotated_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()